EXTRACT IMAGES FROM EXCEL

In [1]:
import openpyxl
import os
from PIL import Image
from io import BytesIO
import re

# Load the Excel file
excel_path = "EXCELS/Cables/Cables_Original.xlsx"
output_folder = "EXCELS/Cables/Cables_images"
os.makedirs(output_folder, exist_ok=True)

# Open workbook
wb = openpyxl.load_workbook(excel_path, data_only=True)
sheet = wb.active

# Function to sanitize the fil
# \ename
def sanitize_filename(name):
    # Replace invalid characters with underscores or remove them
    name = re.sub(r'[<>:"/\\|?*]', '_', name)  # Replace invalid characters
    name = name.replace("\n", "_")  # Replace newline characters with underscore
    name = name.replace("\r", "_")  # Replace carriage return characters with underscore
    return name

# Extract images
image_map = {}  # Store row number → image file path mapping
image_index = 1

for img in sheet._images:
    if isinstance(img, openpyxl.drawing.image.Image):
        # Extract image data correctly
        img_bytes = img._data()
        
        # Convert to PIL image
        image_stream = BytesIO(img_bytes)
        pil_img = Image.open(image_stream)

        # Get the name from a column (e.g., column A is assumed here, adjust as needed)
        name_cell = sheet.cell(row=image_index + 1, column=1)  # Adjust column as needed
        image_name = f"{name_cell.value}.png"  # Use the value in the cell as the name

        # Sanitize the image name
        sanitized_name = sanitize_filename(image_name)

        # Convert image to RGB if it's in CMYK mode
        if pil_img.mode == 'CMYK':
            pil_img = pil_img.convert('RGB')

        # Save the image with the new sanitized name
        image_path = os.path.join(output_folder, sanitized_name)
        pil_img.save(image_path)


        # Store mapping (assuming images align with row numbers)
        row_number = image_index + 1  # Adjust row mapping if needed
        image_map[row_number] = image_path
        image_index += 1

print(f"Extracted {len(image_map)} images to {output_folder}")
print(image_map)


Extracted 477 images to EXCELS/Cables/Cables_images
{2: 'EXCELS/Cables/Cables_images\\Black Cat6 550MHz RJ45 Network Patch Cable (Bootless).png', 3: 'EXCELS/Cables/Cables_images\\Blue Cat6 Network Patch Cable Bootless 550mhz.png', 4: 'EXCELS/Cables/Cables_images\\White Cat6 550MHz Network Patch Cable (Bootless)_.png', 5: 'EXCELS/Cables/Cables_images\\Grey Cat6 550MHz Network Patch Cable (Bootless).png', 6: 'EXCELS/Cables/Cables_images\\Red Cat6 550MHz Network Patch Cable (Bootless).png', 7: 'EXCELS/Cables/Cables_images\\Green Cat6 550MHz Network Patch Cable (Bootless).png', 8: 'EXCELS/Cables/Cables_images\\Violet Cat6 550MHz Network Patch Cable (Bootless).png', 9: 'EXCELS/Cables/Cables_images\\Orange Cat6 550MHz Network Patch Cable (Bootless).png', 10: 'EXCELS/Cables/Cables_images\\Cat6 550MHz Network Patch Cable Aqua (Bootless).png', 11: 'EXCELS/Cables/Cables_images\\Cat6 550MHz Network Patch Cable Brown (Bootless).png', 12: 'EXCELS/Cables/Cables_images\\Cat6 550MHz RJ45 Network Patch

FOR FOLDER

In [31]:
import openpyxl
import os
from PIL import Image
from io import BytesIO
import re

# Path to the folder containing your Excel files
excel_folder = r"EXCELS\Rack\Rack Vendors"
output_folder = r"EXCELS\Rack\Rack_images"
os.makedirs(output_folder, exist_ok=True)

# Function to sanitize the filename
def sanitize_filename(name):
    name = re.sub(r'[<>:"/\\|?*]', '_', name)
    name = name.replace("\n", "_")
    name = name.replace("\r", "_")
    return name

# Function to extract images from a single Excel file
def extract_images_from_excel(excel_path):
    wb = openpyxl.load_workbook(excel_path, data_only=True)
    sheet = wb.active
    image_map = {}
    image_index = 1

    for img in sheet._images:
        if isinstance(img, openpyxl.drawing.image.Image):
            # Extract image data
            img_bytes = img._data()

            # Convert to PIL image
            image_stream = BytesIO(img_bytes)
            pil_img = Image.open(image_stream)

            # Get the name from a column (assume column A here)
            name_cell = sheet.cell(row=image_index + 1, column=1)
            image_name = f"{name_cell.value}.png"  # Name the image based on the cell value

            # Sanitize the image name
            sanitized_name = sanitize_filename(image_name)

            # Convert image to RGB if in CMYK mode
            if pil_img.mode == 'CMYK':
                pil_img = pil_img.convert('RGB')

            # Save the image with the sanitized name
            image_path = os.path.join(output_folder, sanitized_name)
            pil_img.save(image_path)

            # Store the image mapping (row -> image path)
            row_number = image_index + 1
            image_map[row_number] = image_path
            image_index += 1

    return image_map

# Loop through all Excel files in the folder
image_maps = {}

for excel_file in os.listdir(excel_folder):
    if excel_file.endswith(".xlsx"):
        excel_path = os.path.join(excel_folder, excel_file)
        print(f"Processing {excel_file}...")
        image_map = extract_images_from_excel(excel_path)
        image_maps[excel_file] = image_map

print(f"Processed all files in {excel_folder}")


Processing Black Box.xlsx...
Processing Eaton Corporation.xlsx...
Processing Finen.xlsx...
Processing Great Lakes Data Racks & Cabinets.xlsx...
Processing Hewlett Packard Enterprise(HPE).xlsx...
Processing Kendall Howard.xlsx...
Processing Legrand.xlsx...
Processing NETRACK.xlsx...
Processing Schneider.xlsx...
Processed all files in EXCELS\Rack\Rack Vendors


MODIFY THE EXCEL WITH THE PATHS

In [2]:


import os
import pandas as pd

# Define the folder path where images are stored
folder_path = "EXCELS/Cables/cable_images"

# Define the output file path
output_file_path = "EXCELS/Cables/cable.xlsx"

# Check if the output directory exists, and create it if not
output_dir = os.path.dirname(output_file_path)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Load the uploaded Excel file
df = pd.read_excel("EXCELS/Cables/Cables_Original.xlsx", engine='openpyxl')

# Update the 'Image' column with the full file paths
df['Image'] = df['Name'].apply(lambda name: os.path.join(folder_path, f"{name}.png"))

# Save the updated DataFrame back to Excel
df.to_excel(output_file_path, index=False, engine='openpyxl')

print(f"File has been saved successfully to {output_file_path}")



File has been saved successfully to EXCELS/Cables/cable.xlsx


In [6]:
import os
import pandas as pd

# Define the folder paths
input_folder_path = "F:/UI/EXCELS/Rack/Rack Vendors"  # Folder containing the Excel files
image_folder_path = "F:/UI/EXCELS/Rack/Rack_images"  # Folder containing the images
output_folder_path = "F:/UI/EXCELS/Rack/Rack_updated"  # Folder where updated Excel files will be saved

# Ensure the output directory exists
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

# Loop through all Excel files in the input folder
for excel_file in os.listdir(input_folder_path):
    if excel_file.endswith('.xlsx'):
        # Construct full file paths
        input_file_path = os.path.join(input_folder_path, excel_file)
        output_file_path = os.path.join(output_folder_path, excel_file)
        
        # Load the Excel file
        df = pd.read_excel(input_file_path, engine='openpyxl')
        
        # Update the 'Image' column with the full file paths
        df['Image'] = df['Name'].apply(lambda name: os.path.join(image_folder_path, f"{name}.png"))
        
        # Save the updated DataFrame back to Excel
        df.to_excel(output_file_path, index=False, engine='openpyxl')
        
        print(f"Updated file saved: {output_file_path}")


Updated file saved: F:/UI/EXCELS/Rack/Rack_updated\Black Box.xlsx
Updated file saved: F:/UI/EXCELS/Rack/Rack_updated\Eaton Corporation.xlsx
Updated file saved: F:/UI/EXCELS/Rack/Rack_updated\Finen.xlsx
Updated file saved: F:/UI/EXCELS/Rack/Rack_updated\Great Lakes Data Racks & Cabinets.xlsx
Updated file saved: F:/UI/EXCELS/Rack/Rack_updated\Hewlett Packard Enterprise(HPE).xlsx
Updated file saved: F:/UI/EXCELS/Rack/Rack_updated\Kendall Howard.xlsx
Updated file saved: F:/UI/EXCELS/Rack/Rack_updated\Legrand.xlsx
Updated file saved: F:/UI/EXCELS/Rack/Rack_updated\NETRACK.xlsx
Updated file saved: F:/UI/EXCELS/Rack/Rack_updated\Schneider.xlsx


In [3]:
import pandas as pd
import os

# Define the folder containing the Excel files
folder_path = r"EXCELS\Rack\Rack_updated"  # Change this to your actual folder path

# List all Excel files in the folder
files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx') or f.endswith('.xls')]

# Initialize an empty DataFrame
combined_data = pd.DataFrame()

# Loop through each file and append its data to the combined DataFrame
for file in files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_excel(file_path)
    combined_data = pd.concat([combined_data, df], ignore_index=True)

# Export the combined data to a new Excel file
output_path = os.path.join(folder_path, 'Rack.xlsx')
combined_data.to_excel(output_path, index=False)

print(f'Combined Excel file saved at: {output_path}')


Combined Excel file saved at: EXCELS\Rack\Rack_updated\Rack.xlsx


In [19]:

import os
import pandas as pd
import shutil
import re

# Define paths


import os
import pandas as pd
import re
import shutil

# --- CONFIGURE THESE PATHS ---
excel_path = r'F:\UI\EXCELS\Switch\Switches.xlsx'
images_folder = r'F:\UI\EXCELS\Switch\Switch_images'
output_folder = r'F:\UI\EXCELS\Switch\Renamed_kimages' 

# --- Normalization function ---
def normalize(name):
    return re.sub(r'\W+', '', str(name).lower().strip())

# --- Sanitization function for filenames ---
def sanitize_filename(name):
    name = str(name)  # Ensure it's a string
    name = name.replace('\n', ' ').replace('\r', ' ')
    return re.sub(r'[\\/*?:"<>|]', '', name).strip()

# --- Read Excel file ---
df = pd.read_excel(excel_path)

# Make sure 'Name' column exists
if 'Name' not in df.columns:
    raise ValueError("Excel file must contain a 'Name' column.")

# Normalize names from Excel into a dictionary {normalized_name: original_name}
name_map = {normalize(name): name for name in df['Name'] if pd.notnull(name)}

# --- Process and rename images ---
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for filename in os.listdir(image_folder):
    base, ext = os.path.splitext(filename)
    normalized_img = normalize(base)
    match = name_map.get(normalized_img)

    if match:
        clean_name = sanitize_filename(match) + ext.lower()
        output_path = os.path.join(output_folder, clean_name)
        try:
            shutil.copy2(os.path.join(image_folder, filename), output_path)
            print(f"Renamed and copied: {filename} → {clean_name}")
        except Exception as e:
            print(f"Error copying {filename}: {e}")
    else:
        print(f"No match for: {filename}")


Renamed and copied: 16801.png → 16801.png
Renamed and copied: 16802.png → 16802.png
Renamed and copied: 16803.png → 16803.png
Renamed and copied: 16804.png → 16804.png
Renamed and copied: 2526T-PWRAL250011-E6.png → 2526T-PWRAL250011-E6.png
Renamed and copied: 2526T-PWRAL251511-E6.png → 2526T-PWRAL251511-E6.png
Renamed and copied: 2526TAL250001-E6.png → 2526TAL250001-E6.png
Renamed and copied: 2526tal250001e6.png → 2526TAL250001-E6.png
Renamed and copied: 2526TAL251501-E6.png → 2526TAL251501-E6.png
Renamed and copied: 2526tal251501e6.png → 2526TAL251501-E6.png
Renamed and copied: 2526tpwral250011e6.png → 2526T-PWRAL250011-E6.png
Renamed and copied: 2526tpwral251511e6.png → 2526T-PWRAL251511-E6.png
Renamed and copied: 2550T AL250002-E6.png → 2550T AL250002-E6.png
Renamed and copied: 2550T-PWRAL250012-E6.png → 2550T-PWRAL250012-E6.png
Renamed and copied: 2550T-PWRAL251512-E6.png → 2550T-PWRAL251512-E6.png
Renamed and copied: 2550tal250002e6.png → 2550T AL250002-E6.png
Renamed and copied: 

In [26]:


# Create list of expected image filenames
import os
import pandas as pd
import re

# Function to normalize strings for comparison
def normalize(name):
    # Lowercase
    name = name.lower()
    # Remove spaces, underscores, dashes, and other non-alphanumeric chars
    name = re.sub(r'[\s_\-\.\(\)]', '', name)
    return name

# Path to your Excel file and images folder
excel_path = r'F:\UI\EXCELS\Switch\Switches.xlsx'  # Change this to your actual Excel file path
images_folder =  r'F:\UI\EXCELS\Switch\Switch_images'

# Read Excel
df = pd.read_excel(excel_path)

# Assuming the column with names is exactly 'Name'
excel_names = df['Name'].dropna().astype(str).tolist()

# Normalize Excel names
normalized_excel_names = {normalize(name): name for name in excel_names}

# Get all image filenames in the folder (without extension)
image_files = [f for f in os.listdir(images_folder) if f.lower().endswith('.png')]

# Normalize image filenames (remove .png and normalize)
normalized_image_names = set(normalize(os.path.splitext(f)[0]) for f in image_files)

# Find missing files by comparing normalized names
missing_normalized = [norm_name for norm_name in normalized_excel_names if norm_name not in normalized_image_names]

# Get original names of missing files from the dictionary
missing_original_names = [normalized_excel_names[n] for n in missing_normalized]

print(f"Total missing files: {len(missing_original_names)}")
print("Missing files:")
for name in missing_original_names:
    print(name)


Total missing files: 35
Missing files:
XS910/8
HPE Aruba Networking CX 6300M 48p  
SR10 PTP/AVB Class8 PoE 4p 100G  
MACsec Switch (S0E91A) 
HPE Aruba Networking 6300M 24p  
HPE Smart Rate 1G/2.5G/5G/10G  
Class6 PoE and 2p 50G and 2p 25G  
Switch (R8S89A) 
HPE Aruba Networking 6300M 48p  
HPE Smart Rate 1G/2.5G/5G Class8  
PoE and 2p 50G and 2p 25G Switch  
(R8S90A) 
HPE Aruba Networking 6300M 48SR5 12p  

Class8 PoE and 36p Class6 PoE HPE Smart Rate  

1G/2.5G/5G and 2p 50G and 2p 10G LRM support  

Switch (R8S91A) 
HPE Aruba Networking  

6300M 48‑port  

HPE Smart Rate  

1/2.5/5GbE Class 6 PoE  

and 4‑port SFP56 Switch  

(JL659A) 
HPE Aruba Networking  

6300M 24‑port  

HPE Smart Rate  

1/2.5/5GbE Class 6 PoE  

and 4‑port SFP56 Switch  

(JL660A) 
HPE Aruba Networking 6300L 48p Smart  
Rate 1G/2.5G/5G Class8/6 PoE 2p 50G and 2p 10G LRM L2 Switch (S3L77A) 
HPE Aruba Networking 6300L 24p  

Smart Rate 1G/2.5G/5G/10G Class6  

PoE 2p 50G and 2p 25G L2 Switch  

(S3L75A) 
HPE Aru